#Resume Analyser Application

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
df=pd.read_csv('UpdatedResumeDataSet.csv')

: 

In [ ]:
df.head()

: 

In [ ]:
df.tail()

: 

In [ ]:
df.shape

: 

# Exploring Categories

In [ ]:
df['Category']

: 

In [ ]:
df['Category'].value_counts()

: 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data=df, x='Category')
plt.xticks(rotation=90)  # Rotate x-axis labels if needed for better readability
plt.show()


: 

In [ ]:
plt.figure(figsize=(15,10))
count = df['Category'].value_counts()
mylabels=df['Category'].unique()
plt.pie(count,labels=mylabels,autopct='%1.1f%%')
plt.show()

: 

#Exploring Resume

In [ ]:
df['Category'][0]

: 

In [ ]:
df['Resume'][0]

: 

#Cleaning Data:
1.URLs

2.Hashtags

3.Mentions

4.Special Letters

5.Punctuations

In [ ]:
import re
def cleaning(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    pattern = re.compile(r'@\S+')
    pattern = re.compile(r'RT|cc')
    pattern = re.compile(r'#\S+')
    pattern = re.compile(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""))
    pattern = re.compile(r'[^\x00-\x7f]')
    pattern = re.compile(r'\s+')
    return pattern.sub(r'', text)


: 

In [ ]:
cleaning(df['Resume'][0])

: 

#Words into categorical values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

: 

In [ ]:
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

: 

In [ ]:
df.Category.unique()

: 

#Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')

tfidf.fit(df['Resume'])
requiredText=tfidf.transform(df['Resume'])

: 

In [ ]:
df

: 

#Splitting


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(requiredText, df['Category'], test_size=0.2, random_state=1)

: 

In [ ]:
X_train.shape

: 

In [ ]:
X_test.shape

: 

#Now let's train the model and print the classification report

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

: 

In [ ]:
clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
ypred = clf.predict(X_test)
print(accuracy_score(y_test,ypred))

: 

In [ ]:
print(ypred)

: 

#Prediction System

In [ ]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(clf, open('clf.pkl', 'wb'))

: 

In [ ]:
pip install PyPDF2

: 

In [ ]:
import PyPDF2

# Open the PDF file
with open('/content/Jit saha_new.pdf', 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Initialize an empty string to store the extracted text
    myresume = ''

    # Iterate through each page and extract text
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        myresume += page.extract_text()

# Now, 'myresume' contains the text extracted from the PDF file


: 

In [ ]:
import pickle

# Load the trained classifier
clf = pickle.load(open('clf.pkl', 'rb'))

# Clean the input resume
cleaned_resume = cleaning(myresume)

# Transform the cleaned resume using the trained TfidfVectorizer
input_features = tfidf.transform([cleaned_resume])

# Make the prediction using the loaded classifier
prediction_id = clf.predict(input_features)[0]

# Map category ID to category name
category_mapping = {
    15: "Java Developer",
    23: "Testing",
    8: "DevOps Engineer",
    20: "Python Developer",
    24: "Web Designing",
    12: "HR",
    13: "Hadoop",
    3: "Blockchain",
    10: "ETL Developer",
    18: "Operations Manager",
    6: "Data Science",
    22: "Sales",
    16: "Mechanical Engineer",
    1: "Arts",
    7: "Database",
    11: "Electrical Engineering",
    14: "Health and fitness",
    19: "PMO",
    4: "Business Analyst",
    9: "DotNet Developer",
    2: "Automation Testing",
    17: "Network Security Engineer",
    21: "SAP Developer",
    5: "Civil Engineer",
    0: "Advocate",
}

category_name = category_mapping.get(prediction_id, "Unknown")

print("Predicted Category:", category_name)
print(prediction_id)

: 